In [13]:
import torch
import transformers
from transformers import (
    LlamaTokenizer,
    LlamaForCausalLM,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    Trainer,
    GenerationConfig
)
import accelerate
import bitsandbytes
from tqdm import tqdm
from trl import SFTTrainer
import time
import pandas as pd
import numpy as np
import datasets

from data.preprocess_dataset import preprocess_dataset

# Loading and processing dataset

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained("maritaca-ai/sabia-7b")
tokenizer.add_special_tokens({"pad_token": "<PAD>"})
#tokenizer.save_pretrained("./data/custom_tokenizer")

canarim_dataset_name = "dominguesm/Canarim-Instruct-PTBR-Dataset"
dataset = datasets.load_dataset(canarim_dataset_name)

In [ ]:
process_time = time.time()
print("Starting dataset processing...")
processed_dataset = preprocess_dataset(dataset, tokenizer)
print(f"Finished processing. It took {time.time() - process_time} seconds\n")

processed_dataset

# Loading Sabia7B Model

In [4]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
)

In [14]:
model_name = "maritaca-ai/sabia-7b"

model = LlamaForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16   # If your GPU does not support bfloat16, change to torch.float16
)

# Tá dando erro pq eu acho que não consigo ativar o CUDA localmente (minha GPU é AMD) (Jonas)
model

ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [16]:
torch.cuda.is_available()

False